In [1]:
import seaborn as sns
import pandas as pd
import scipy.stats as st
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import warnings
from statistics import mean
import ast

warnings.filterwarnings('ignore')

# RQ1/2: Measurement of Quality 

In [24]:
files = os.listdir(os.path.join('.', 'quality/max10itemsperstack'))
mean = pd.DataFrame(columns = ["Option", "Initial Plan", "Initial Plan after Disturbance", "Naive Repair", "Replanning", "Replanning (seed 0.2)"])
std = pd.DataFrame(columns = ["Option", "Naive Repair", "Replanning", "Replanning (seed 0.2)"])
for f in files:
    if f.endswith("csv"):
        df = pd.read_csv(os.path.join('.', 'quality/max10itemsperstack', f))
        mean_row = {}
        std_row = {}
        for name, group in df.groupby(by='variant'):
            if(name == "NSGAII_NaivePlanningStrategy_2"):
                mean_row["Naive Repair"] = round(group['final_objective_value'].mean(), 3)
                mean_row["Initial Plan"] = round(group['initial_planning_objective'].mean(), 3)
                mean_row["Initial Plan after Disturbance"] = round(group['initial_post_disturbance_std'].mean(), 3)
                std_row["Naive Repair"] = round(group['final_objective_value'].std(), 3)
            elif(name == "NSGAII_EvaluationReplanningStrategy-10000_1" or name == "NSGAII_EvaluationReplanningStrategy-10000_0"):
                mean_row["Replanning"] = round(group['final_objective_value'].mean(), 3)
                std_row["Replanning"] = round(group['final_objective_value'].std(), 3)
            elif(name == "NSGAII_EvaluationReplanningStrategy-10000reusePortion=0.2_withReinit_0" or name == "NSGAII_EvaluationReplanningStrategy-10000reusePortion=0.2_withSeed_1"):
                mean_row["Replanning (seed 0.2)"] = round(group['final_objective_value'].mean(), 3)
                std_row["Replanning (seed 0.2)"] = round(group['final_objective_value'].std(), 3)
           
        mean_row["Option"] = f.split("@")[1].split(".")[0]
        mean = mean.append(mean_row, ignore_index = True)
        std_row["Option"] = f.split("@")[1].split(".")[0]
        std = std.append(std_row, ignore_index = True)
            # todo: median
            # todo: variance
print("Mean - Absolute Values")
display(mean)

print("Standard Deviation - Absolut Values")
display(std)
#print(std)
# todo: create table for each rq
# todo: create plot for each rq

# todo: in den CSVs: naive Planning -> naive Repair



Mean - Absolute Values


,Option,Initial Plan,Initial Plan after Disturbance,Naive Repair,Replanning,Replanning (seed 0.2)
0,ADD_STACKS_FIRST_10_PERCENT_errPerDisturbance-5,1.693,3.360,2.202,2.017,2.171
1,ADD_STACKS_LAST_10_PERCENT_errPerDisturbance-5,1.713,2.315,2.182,1.710,2.163
2,ADD_STACKS_MIDDLE_10_PERCENT_errPerDisturbance-5,1.744,3.027,2.226,1.881,2.169
3,REMOVE_STACKS_FIRST_10_PERCENT_errPerDisturban...,1.717,3.108,1.839,1.615,1.740
4,REMOVE_STACKS_LAST_10_PERCENT_errPerDisturbance-5,1.715,1.890,1.742,1.386,1.702
5,REMOVE_STACKS_MIDDLE_10_PERCENT_errPerDisturba...,1.715,2.754,1.803,1.470,1.755


Standard Deviation - Absolut Values


,Option,Naive Repair,Replanning,Replanning (seed 0.2)
0,ADD_STACKS_FIRST_10_PERCENT_errPerDisturbance-5,0.106,0.098,0.091
1,ADD_STACKS_LAST_10_PERCENT_errPerDisturbance-5,0.125,0.116,0.084
2,ADD_STACKS_MIDDLE_10_PERCENT_errPerDisturbance-5,0.119,0.090,0.111
3,REMOVE_STACKS_FIRST_10_PERCENT_errPerDisturban...,0.153,0.145,0.115
4,REMOVE_STACKS_LAST_10_PERCENT_errPerDisturbance-5,0.141,0.128,0.162
5,REMOVE_STACKS_MIDDLE_10_PERCENT_errPerDisturba...,0.142,0.139,0.208


In [3]:
mean_relative = pd.DataFrame(columns = ["Option", "Diff Initial after Disturbance", "Diff Naive Repair to Initial", 
                                       "Diff Replanning to Naive Repair", "Diff seed 0.2 to no seed"])
mean_relative["Diff Initial after Disturbance"] = (mean["Initial Plan after Disturbance"] - mean["Initial Plan"]) / mean["Initial Plan"] * 100
mean_relative["Diff Naive Repair to Initial"] = (mean["Naive Repair"] - mean["Initial Plan"]) / mean["Initial Plan"] * 100
mean_relative["Diff Replanning to Naive Repair"] = (mean["Replanning"] - mean["Naive Repair"])/ mean["Naive Repair"] * 100
mean_relative["Diff seed 0.2 to no seed"] = (mean["Replanning (seed 0.2)"] - mean["Replanning"])/ mean["Replanning"] * 100
mean_relative["Option"] = mean["Option"]
mean_relative.round(3)
display(mean_relative)

,Option,Diff Initial after Disturbance,Diff Naive Repair to Initial,Diff Replanning to Naive Repair,Diff seed 0.2 to no seed
0,ADD_STACKS_FIRST_10_PERCENT_errPerDisturbance-5,98.464265,30.064973,-8.401453,7.635102
1,ADD_STACKS_LAST_10_PERCENT_errPerDisturbance-5,35.143024,27.378867,-21.631531,26.491228
2,ADD_STACKS_MIDDLE_10_PERCENT_errPerDisturbance-5,73.566514,27.637615,-15.498652,15.311005
3,REMOVE_STACKS_FIRST_10_PERCENT_errPerDisturban...,81.013395,7.105416,-12.180533,7.739938
4,REMOVE_STACKS_LAST_10_PERCENT_errPerDisturbance-5,10.204082,1.574344,-20.436280,22.799423
5,REMOVE_STACKS_MIDDLE_10_PERCENT_errPerDisturba...,60.583090,5.131195,-18.469218,19.387755


# RQ2: Measurement of Evaluations/Time
### Predefined standard deviation ( >= 50% decrease from initial std); Search until quality is sufficient (for initial plan & when replanning after disturbance)


In [23]:
files = os.listdir(os.path.join('.', 'threshold_rq1_rq2', '1to100'))
mean = pd.DataFrame(columns = ["Option", "Replanning", "Replanning (seed 0.2)", "Diff to no seed"])
std = pd.DataFrame(columns = ["Option", "Replanning", "Replanning (seed 0.2)"])
for f in files:
    if f.endswith("csv"):
        df = pd.read_csv(os.path.join('.', 'threshold_rq1_rq2', '1to100', f))
        mean_row = {}
        std_row = {}
        for name, group in df.groupby(by='variant'):
            if(name == "NSGAII_PlanningStrategy-ReplanningStrategy_0"):
                mean_row["Replanning"] = round(group['last_replanning_evaluations'].mean(), 3)
                std_row["Replanning"] = round(group['last_replanning_evaluations'].std(), 3)
            elif(name == "NSGAII_PlanningStrategy-ReplanningStrategy_reuse0.10_1"):
                mean_row["Replanning (seed 0.2)"] = round(group['last_replanning_evaluations'].mean(), 3)
                std_row["Replanning (seed 0.2)"] = round(group['last_replanning_evaluations'].std(), 3)
           
        #print(f)"NSGAII_PlanningStrategy-ReplanningStrategy_reuse0.10_1" und "NSGAII_PlanningStrategy-ReplanningStrategy_0"
       # print(mean_row)
        mean_row["Option"] = f.split("@")[1].split("_errPerDisturbance")[0]
        mean = mean.append(mean_row, ignore_index = True)
        std_row["Option"] = f.split("@")[1].split(".")[0]
        std = std.append(std_row, ignore_index = True)
            # todo: median
            # todo: variance
print("Mean - Replanning time")
mean["Diff to no seed"] = (mean["Replanning (seed 0.2)"] - mean["Replanning"]) / mean["Replanning"] * 100
display(mean)

print("Standard Deviation - Replanning time")
display(std)
#print(std)
# todo: create table for each rq
# todo: create plot for each rq

# todo: in den CSVs: naive Planning -> naive Repair



Mean - Replanning time


,Option,Replanning,Replanning (seed 0.2),Diff to no seed
0,ADD_STACKS_FIRST_10_PERCENT,68976.667,370846.667,437.640746
1,ADD_STACKS_MIDDLE_10_PERCENT,36686.667,595290.000,1522.633094
2,ADD_STACKS_LAST_10_PERCENT,9400.000,1723260.000,18232.553191
3,REMOVE_STACKS_LAST_10_PERCENT,6320.000,18636.667,194.883972
4,REMOVE_STACKS_FIRST_10_PERCENT,12363.333,11786.667,-4.664325
5,REMOVE_STACKS_MIDDLE_10_PERCENT,9563.333,57460.000,500.836549


Standard Deviation - Replanning time


,Option,Replanning,Replanning (seed 0.2)
0,ADD_STACKS_FIRST_10_PERCENT_errPerDisturbance-5,219068.218,715830.393
1,ADD_STACKS_MIDDLE_10_PERCENT_errPerDisturbance-5,96116.003,875837.930
2,ADD_STACKS_LAST_10_PERCENT_errPerDisturbance-5,3143.686,899061.889
3,REMOVE_STACKS_LAST_10_PERCENT_errPerDisturbance-5,1453.035,86310.812
4,REMOVE_STACKS_FIRST_10_PERCENT_errPerDisturban...,4928.208,19721.068
5,REMOVE_STACKS_MIDDLE_10_PERCENT_errPerDisturba...,2628.292,160589.345


### Test with decreasing solution lengths => solutionLength = 200 - nrOfExecutedSteps 
* nr. runs < 30

#### Mean

In [20]:
files = os.listdir(os.path.join('.', 'threshold_rq1_rq2', '1to100_withdecreaseInPlanSize'))
mean = pd.DataFrame(columns = ["Option", "Replanning", "Replanning (seed 0.2)", "Diff to no seed"])
std = pd.DataFrame(columns = ["Option", "Replanning", "Replanning (seed 0.2)"])
for f in files:
    if f.endswith("csv"):
        df = pd.read_csv(os.path.join('.', 'threshold_rq1_rq2', '1to100_withdecreaseInPlanSize',f), sep=",")
        mean_row = {}
        std_row = {}
        for name, group in df.groupby(by='variant'):
            if(name == "NSGAII_PlanningStrategy-ReplanningStrategy_0"):
                mean_row["Replanning"] = round(group['last_replanning_evaluations'].mean(), 3)
                std_row["Replanning"] = round(group['last_replanning_evaluations'].std(), 3)
            elif(name == "NSGAII_PlanningStrategy-ReplanningStrategy_reuse0.10_1"):
                mean_row["Replanning (seed 0.2)"] = round(group['last_replanning_evaluations'].mean(), 3)
                std_row["Replanning (seed 0.2)"] = round(group['last_replanning_evaluations'].std(), 3)
           
        #print(f)"NSGAII_PlanningStrategy-ReplanningStrategy_reuse0.10_1" und "NSGAII_PlanningStrategy-ReplanningStrategy_0"
       # print(mean_row)
        mean_row["Option"] = f.split("@")[1].split("_errPerDisturbance")[0]
        mean = mean.append(mean_row, ignore_index = True)
        std_row["Option"] = f.split("@")[1].split(".")[0]
        std = std.append(std_row, ignore_index = True)
            # todo: median
            # todo: variance
print("Mean - Replanning time")
mean["Diff to no seed"] = (mean["Replanning (seed 0.2)"] - mean["Replanning"]) / mean["Replanning"] * 100
display(mean)

print("Standard Deviation - Replanning time")
display(std)
#print(std)
# todo: create table for each rq
# todo: create plot for each rq

# todo: in den CSVs: naive Planning -> naive Repair



Mean - Replanning time


,Option,Replanning,Replanning (seed 0.2),Diff to no seed
0,REMOVE_STACKS_FIRST_10_PERCENT,14750.000,6295.0,-57.322034
1,REMOVE_STACKS_MIDDLE_10_PERCENT,8420.000,70820.0,741.092637
2,REMOVE_STACKS_LAST_10_PERCENT,5553.333,7820.0,40.816335


Standard Deviation - Replanning time


,Option,Replanning,Replanning (seed 0.2)
0,REMOVE_STACKS_FIRST_10_PERCENT_errPerDisturban...,8142.255,9078.400
1,REMOVE_STACKS_MIDDLE_10_PERCENT_errPerDisturba...,2386.031,224425.359
2,REMOVE_STACKS_LAST_10_PERCENT_errPerDisturbance-5,1588.740,21551.904


#### Median

In [21]:
files = os.listdir(os.path.join('.', 'threshold_rq1_rq2', '1to100_withdecreaseInPlanSize'))
mean = pd.DataFrame(columns = ["Option", "Replanning", "Replanning (seed 0.2)", "Diff to no seed"])
std = pd.DataFrame(columns = ["Option", "Replanning", "Replanning (seed 0.2)"])
for f in files:
    if f.endswith("csv"):
        df = pd.read_csv(os.path.join('.', 'threshold_rq1_rq2', '1to100_withdecreaseInPlanSize',f), sep=",")
        mean_row = {}
        std_row = {}
        for name, group in df.groupby(by='variant'):
            if(name == "NSGAII_PlanningStrategy-ReplanningStrategy_0"):
                mean_row["Replanning"] = round(group['last_replanning_evaluations'].median(), 3)
                std_row["Replanning"] = round(group['last_replanning_evaluations'].std(), 3)
            elif(name == "NSGAII_PlanningStrategy-ReplanningStrategy_reuse0.10_1"):
                mean_row["Replanning (seed 0.2)"] = round(group['last_replanning_evaluations'].median(), 3)
                std_row["Replanning (seed 0.2)"] = round(group['last_replanning_evaluations'].std(), 3)
           
        #print(f)"NSGAII_PlanningStrategy-ReplanningStrategy_reuse0.10_1" und "NSGAII_PlanningStrategy-ReplanningStrategy_0"
       # print(mean_row)
        mean_row["Option"] = f.split("@")[1].split("_errPerDisturbance")[0]
        mean = mean.append(mean_row, ignore_index = True)
        std_row["Option"] = f.split("@")[1].split(".")[0]
        std = std.append(std_row, ignore_index = True)
            # todo: median
            # todo: variance
print("Median - Replanning time")
mean["Diff to no seed"] = (mean["Replanning (seed 0.2)"] - mean["Replanning"]) / mean["Replanning"] * 100
display(mean)

print("Standard Deviation - Replanning time")
display(std)
#print(std)
# todo: create table for each rq
# todo: create plot for each rq

# todo: in den CSVs: naive Planning -> naive Repair



Median - Replanning time


,Option,Replanning,Replanning (seed 0.2),Diff to no seed
0,REMOVE_STACKS_FIRST_10_PERCENT,12750.0,4200.0,-67.058824
1,REMOVE_STACKS_MIDDLE_10_PERCENT,8700.0,3500.0,-59.770115
2,REMOVE_STACKS_LAST_10_PERCENT,5300.0,400.0,-92.452830


Standard Deviation - Replanning time


,Option,Replanning,Replanning (seed 0.2)
0,REMOVE_STACKS_FIRST_10_PERCENT_errPerDisturban...,8142.255,9078.400
1,REMOVE_STACKS_MIDDLE_10_PERCENT_errPerDisturba...,2386.031,224425.359
2,REMOVE_STACKS_LAST_10_PERCENT_errPerDisturbance-5,1588.740,21551.904


# RQ3: Parallel Replanning
### Predefined standard deviation ( >= 50% decrease from initial std); Search until quality is sufficient (initial plan & replan after disturbance while system idle), search in parallel until Q1 = Q2 and T1 < T2

In [33]:
T = 10
files = os.listdir(os.path.join('.', 'parallel_rq3'))
overall_runtime = pd.DataFrame(columns = ["Option", "Idle Replan", "Parallel (2)", "Parallel (5)", "Parallel (10)"])
overall_runtime_std = pd.DataFrame(columns =  ["Option", "Idle Replan",  "Parallel (2)", "Parallel (5)", "Parallel (10)"])
for f in files:
    if f.endswith("csv"):
        df = pd.read_csv(os.path.join('.', 'parallel_rq3', f), sep=";")
        df_p = pd.read_csv(os.path.join('.', 'parallel_rq3', "parallel_results", f[:-4] + "_predictive.csv"), sep=";")

        ## reenumerate predictive run rows from 0,1,2,3,4,5 to 0,0,0, 1,1,1, ...
        df_p['run'] = df_p['run'].map(int)/3
        df_p["run"] = df_p["run"].map(int)
        
        step_variants = df_p["predicted_steps"].unique()
        
        ## get idle computed objs from predictive run file
        idle_computed_objs = []
        for idx, row in df.iterrows():
            #if not df_p.loc[df_p["run"] == row['run']].empty:
                #idle_computed_objs
                #print(df_p.loc[df_p["run"] == row['run']].iloc[0]["idle_computed_objectives"])
            idle_computed_objs.append(df_p.loc[df_p["run"] == row['run']].iloc[0]["idle_computed_objectives"])
 
        df["idle_computed_objectives"] = idle_computed_objs
        df["idle_computed_steps"] = [ast.literal_eval(g)[1] for g in idle_computed_objs]
        df["idle_computed_steps_execution_time"] = [ast.literal_eval(g)[1]*T for g in idle_computed_objs]
        df["overall_execution_time"] =  df["idle_computed_steps_execution_time"] + df["last_replanning_time"]
       # print(df)
        mean_executiontime_row = {}
        std_executiontime_row = {}

        df_p["pred_plan_steps"] = [ast.literal_eval(g)[1] for g in df_p["pred_plan_objectives"]]
        df_p["pred_plan_steps_execution_time"] = [ast.literal_eval(g)[1]*T for g in df_p["pred_plan_objectives"]]
        df_p["pred_overall_execution_time"] =  df_p["pred_plan_steps_execution_time"] + df_p["pred_plan_runtime"]

        for name, g in df.groupby(by='variant'):
            mean_executiontime_row["Idle Replan"] = round(g['overall_execution_time'].median(),3)
            std_executiontime_row["Idle Replan"] = round(g['overall_execution_time'].std(),3)/mean_executiontime_row["Idle Replan"]*100            
            
        for name, group in df_p.groupby(by=['variant', 'predicted_steps']):
            mean_executiontime_row[f"Parallel ({name[1]})"] = round(group['pred_overall_execution_time'].median(), 3)
            std_executiontime_row[f"Parallel ({name[1]})"] = round(group['pred_overall_execution_time'].std(), 3)/mean_executiontime_row[f"Parallel ({name[1]})"]*100
   
      
        mean_executiontime_row["Option"] = f.split("@")[1].split("_errPerDisturbance")[0]
        std_executiontime_row["Option"] = f.split("@")[1].split("_errPerDisturbance")[0]
        overall_runtime = overall_runtime.append(mean_executiontime_row, ignore_index = True)
        overall_runtime_std = overall_runtime_std.append(std_executiontime_row, ignore_index = True)

        
print("Mean - Execution time (seconds)")
display(overall_runtime.iloc[:2, :])
display(overall_runtime.iloc[2:, :])

for steps in step_variants:
    overall_runtime[f"Parallel ({steps})"] = (overall_runtime[f"Parallel ({steps})"] - overall_runtime["Idle Replan"])/ overall_runtime["Idle Replan"] * 100

#mean.round({'Replanning': 1})
#mean["Replanning (seed 0.2)"] = (mean["Replanning (seed 0.2)"] - mean["Replanning"])/ mean["Replanning"] * 100
#mean.round(3)
#print("Mean - Replanning time (with seed) relative to plain replanning (in %)")
print("Mean - Execution time relative to idle replanning (%)")
display(overall_runtime.iloc[:2, :])
display(overall_runtime.iloc[2:, :])

print("Standard Deviation - Execution time (seconds)")
#overall_runtime_std.iloc[:, 1:] = overall_runtime_std.iloc[:, 1:]/overall_runtime.iloc[:, 1:]


display(overall_runtime_std.iloc[:2, :])
display(overall_runtime_std.iloc[2:, :])
# todo: create table for each rq
# todo: create plot for each rq

# todo: in den CSVs: naive Planning -> naive Repair

Mean - Execution time (seconds)


,Option,Idle Replan,Parallel (2),Parallel (5),Parallel (10)
0,ADD_STACKS_FIRST_10_PERCENT,521.915,545.965,609.744,425.146
1,ADD_STACKS_MIDDLE_10_PERCENT,588.101,502.324,512.091,420.587


,Option,Idle Replan,Parallel (2),Parallel (5),Parallel (10)
2,REMOVE_STACKS_FIRST_10_PERCENT,545.351,538.186,467.768,290.092
3,REMOVE_STACKS_MIDDLE_10_PERCENT,585.096,598.230,510.206,240.115


Mean - Execution time relative to idle replanning (%)


,Option,Idle Replan,Parallel (2),Parallel (5),Parallel (10)
0,ADD_STACKS_FIRST_10_PERCENT,521.915,4.60803,16.828219,-18.541142
1,ADD_STACKS_MIDDLE_10_PERCENT,588.101,-14.58542,-12.924651,-28.483883


,Option,Idle Replan,Parallel (2),Parallel (5),Parallel (10)
2,REMOVE_STACKS_FIRST_10_PERCENT,545.351,-1.313833,-14.226251,-46.806369
3,REMOVE_STACKS_MIDDLE_10_PERCENT,585.096,2.244760,-12.799609,-58.961435


Standard Deviation - Execution time (seconds)


,Option,Idle Replan,Parallel (2),Parallel (5),Parallel (10)
0,ADD_STACKS_FIRST_10_PERCENT,31.159480,28.890130,94.376984,44.945266
1,ADD_STACKS_MIDDLE_10_PERCENT,26.516194,38.470589,34.026179,49.124676


,Option,Idle Replan,Parallel (2),Parallel (5),Parallel (10)
2,REMOVE_STACKS_FIRST_10_PERCENT,36.016254,28.628207,34.489533,22.299822
3,REMOVE_STACKS_MIDDLE_10_PERCENT,30.208547,41.244672,47.131943,49.538763
